In [1]:
import numpy as np
import pandas as pd

In [2]:
emart_all = pd.read_excel("../데이터/이마트 총합 데이터.xlsx")
emart_all

,지점명,주소,3km내 이마트,3km내 경쟁 대형마트,3km내 창고형 마트,부지 면적,부지 공시지가,주차 시설,대중 교통,점포형태
0,청계천점,대한민국 서울특별시 중구 황학동 청계천로 400,1,3,0,96410.82,861912730800,1000.0,NaN,0
1,용산점,대한민국 서울특별시 용산구 한강로3가 한강대로23길 55,1,0,0,42405.64,641173276800,2100.0,"용산역, 신용산역",0
2,창동점,대한민국 서울특별시 도봉구 창동 노해로65길 4,1,4,0,12776.00,90530736000,159.0,창동역,0
3,미아점,대한민국 서울특별시 성북구 길음동 도봉로 17,2,2,0,20690.71,316154048800,195.0,미아사거리역,0
4,하월곡점,대한민국 서울특별시 성북구 하월곡동 230,2,2,0,75041.00,479511990000,760.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...
162,시지점,대구광역시 수성구 고산로 101 (신매동 566-3),1,2,0,42302.00,162058962000,NaN,NaN,2
163,부평점,인천광역시 부평구 주부토로 241 (갈산동 171),1,8,0,6600.00,12566400000,NaN,NaN,2
164,덕이점,고양시 일산서구 경의로 798 (덕이동 205),2,2,1,31948.83,73642053150,676.0,NaN,2
165,서부산점,부산광역시 사상구 낙동대로 910 (감전동),1,2,0,71773.50,133929351000,984.0,NaN,2


In [3]:
emart_all.corr()

,3km내 이마트,3km내 경쟁 대형마트,3km내 창고형 마트,부지 면적,부지 공시지가,주차 시설,점포형태
3km내 이마트,1.000000,0.431286,0.336891,0.173758,0.468155,0.096230,0.237032
3km내 경쟁 대형마트,0.431286,1.000000,0.398570,0.110471,0.249457,0.102099,0.196347
3km내 창고형 마트,0.336891,0.398570,1.000000,0.213024,0.337246,0.102404,-0.065661
부지 면적,0.173758,0.110471,0.213024,1.000000,0.677824,0.866375,0.202284
부지 공시지가,0.468155,0.249457,0.337246,0.677824,1.000000,0.532539,-0.001297
주차 시설,0.096230,0.102099,0.102404,0.866375,0.532539,1.000000,0.189177
점포형태,0.237032,0.196347,-0.065661,0.202284,-0.001297,0.189177,1.000000


In [4]:
parking = emart_all[["부지 면적","부지 공시지가","주차 시설"]]

In [5]:
parking

,부지 면적,부지 공시지가,주차 시설
0,96410.82,861912730800,1000.0
1,42405.64,641173276800,2100.0
2,12776.00,90530736000,159.0
3,20690.71,316154048800,195.0
4,75041.00,479511990000,760.0
...,...,...,...
162,42302.00,162058962000,NaN
163,6600.00,12566400000,NaN
164,31948.83,73642053150,676.0
165,71773.50,133929351000,984.0


In [6]:
parking.loc[parking.isna().sum(1) == True,"주차 시설"] = 0 #결측치 있는 곳만 보기

/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
parking[parking["주차 시설"] == 0]

,부지 면적,부지 공시지가,주차 시설
159,22000.0,30259990320,0.0
162,42302.0,162058962000,0.0
163,6600.0,12566400000,0.0


In [8]:
# 그래서 머신러닝으로 예측을 해서 풍속을 넣어주도록 한다.
from sklearn.ensemble import RandomForestClassifier

def predict_parking_space(data):
    
    # 풍속이 0인것과 아닌 것을 나누어 준다.
    dataPark0 = data.loc[data['주차 시설'] == 0]
    dataParkNot0 = data.loc[data['주차 시설'] != 0]
    
    # 풍속을 예측할 피처를 선택한다.
    pCol = ["부지 면적", "부지 공시지가"]

    # 풍속이 0이 아닌 데이터들의 타입을 스트링으로 바꿔준다.
    dataParkNot0["주차 시설"] = dataParkNot0["주차 시설"].astype("str")

    # 랜덤포레스트 분류기를 사용한다.
    rfModel_park = RandomForestClassifier()

    # wCol에 있는 피처의 값을 바탕으로 풍속을 학습시킨다.
    rfModel_park.fit(dataParkNot0[pCol], dataParkNot0["주차 시설"])

    # 학습한 값을 바탕으로 풍속이 0으로 기록 된 데이터의 풍속을 예측한다.
    park0Values = rfModel_park.predict(X = dataPark0[pCol])

    # 값을 다 예측 후 비교해 보기 위해
    # 예측한 값을 넣어 줄 데이터 프레임을 새로 만든다.
    predictPark0 = dataPark0
    predictParkNot0 = dataParkNot0

    # 값이 0으로 기록 된 풍속에 대해 예측한 값을 넣어준다.
    predictPark0["주차 시설"] = park0Values

    # dataWindNot0 0이 아닌 풍속이 있는 데이터프레임에 예측한 값이 있는 데이터프레임을 합쳐준다.
    data = predictParkNot0.append(predictPark0)

    # 풍속의 데이터타입을 float으로 지정해 준다.
    data["주차 시설"] = data["주차 시설"].astype("float")

    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    
    return data

In [9]:
predict_parking_space(parking)

/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,부지 면적,부지 공시지가,주차 시설
0,96410.82,861912730800,1000.0
1,42405.64,641173276800,2100.0
2,12776.00,90530736000,159.0
3,20690.71,316154048800,195.0
4,75041.00,479511990000,760.0
...,...,...,...
162,71773.50,133929351000,984.0
163,45416.28,143242947120,543.0
164,22000.00,30259990320,620.0
165,42302.00,162058962000,705.0


In [10]:
parking[parking["주차 시설"] == 0]

,부지 면적,부지 공시지가,주차 시설
159,22000.0,30259990320,0.0
162,42302.0,162058962000,0.0
163,6600.0,12566400000,0.0


In [11]:
predict_parking_space(parking).iloc[[159,162,163],:]

/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,부지 면적,부지 공시지가,주차 시설
159,23387.83,94603772350,298.0
162,71773.50,133929351000,984.0
163,45416.28,143242947120,543.0


In [12]:
predict_parking_space(parking).iloc[[159,162,163],:]["주차 시설"]

/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/dongmin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


159    298.0
162    984.0
163    543.0
Name: 주차 시설, dtype: float64

In [13]:
emart_all.loc[159,"주차 시설"]

nan

In [14]:
emart_all.loc[159,"주차 시설"] = 298.0
emart_all.loc[162,"주차 시설"] = 984.0
emart_all.loc[163,"주차 시설"] = 543.0

In [15]:
emart_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   지점명           167 non-null    object 
 1   주소            167 non-null    object 
 2   3km내 이마트      167 non-null    int64  
 3   3km내 경쟁 대형마트  167 non-null    int64  
 4   3km내 창고형 마트   167 non-null    int64  
 5   부지 면적         167 non-null    float64
 6   부지 공시지가       167 non-null    int64  
 7   주차 시설         167 non-null    float64
 8   대중 교통         81 non-null     object 
 9   점포형태          167 non-null    int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 13.2+ KB


In [16]:
# emart_all.to_excel("../데이터/emart_all(주차시설 ML이용 결측치 채움).xlsx",encoding="cp949",index=False)

In [17]:
emart_all.corr()

,3km내 이마트,3km내 경쟁 대형마트,3km내 창고형 마트,부지 면적,부지 공시지가,주차 시설,점포형태
3km내 이마트,1.000000,0.431286,0.336891,0.173758,0.468155,0.091474,0.237032
3km내 경쟁 대형마트,0.431286,1.000000,0.398570,0.110471,0.249457,0.087150,0.196347
3km내 창고형 마트,0.336891,0.398570,1.000000,0.213024,0.337246,0.104516,-0.065661
부지 면적,0.173758,0.110471,0.213024,1.000000,0.677824,0.865694,0.202284
부지 공시지가,0.468155,0.249457,0.337246,0.677824,1.000000,0.534403,-0.001297
주차 시설,0.091474,0.087150,0.104516,0.865694,0.534403,1.000000,0.149892
점포형태,0.237032,0.196347,-0.065661,0.202284,-0.001297,0.149892,1.000000


In [18]:
emart_all.loc[emart_all.isna().sum(1) == True,"대중 교통"] = "0"

In [19]:
emart_all

,지점명,주소,3km내 이마트,3km내 경쟁 대형마트,3km내 창고형 마트,부지 면적,부지 공시지가,주차 시설,대중 교통,점포형태
0,청계천점,대한민국 서울특별시 중구 황학동 청계천로 400,1,3,0,96410.82,861912730800,1000.0,0,0
1,용산점,대한민국 서울특별시 용산구 한강로3가 한강대로23길 55,1,0,0,42405.64,641173276800,2100.0,"용산역, 신용산역",0
2,창동점,대한민국 서울특별시 도봉구 창동 노해로65길 4,1,4,0,12776.00,90530736000,159.0,창동역,0
3,미아점,대한민국 서울특별시 성북구 길음동 도봉로 17,2,2,0,20690.71,316154048800,195.0,미아사거리역,0
4,하월곡점,대한민국 서울특별시 성북구 하월곡동 230,2,2,0,75041.00,479511990000,760.0,0,0
...,...,...,...,...,...,...,...,...,...,...
162,시지점,대구광역시 수성구 고산로 101 (신매동 566-3),1,2,0,42302.00,162058962000,984.0,0,2
163,부평점,인천광역시 부평구 주부토로 241 (갈산동 171),1,8,0,6600.00,12566400000,543.0,0,2
164,덕이점,고양시 일산서구 경의로 798 (덕이동 205),2,2,1,31948.83,73642053150,676.0,0,2
165,서부산점,부산광역시 사상구 낙동대로 910 (감전동),1,2,0,71773.50,133929351000,984.0,0,2


In [26]:
for i in range(len(emart_all["대중 교통"])):
    if "," in emart_all.loc[i,"대중 교통"]:
        emart_all.loc[i,"대중 교통"] = emart_all.loc[i,"대중 교통"].count(",")+1
    elif emart_all.loc[i,"대중 교통"] == "0":
        emart_all.loc[i,"대중 교통"] = 0
    else:
        emart_all.loc[i,"대중 교통"] = 1

In [29]:
emart_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   지점명           167 non-null    object 
 1   주소            167 non-null    object 
 2   3km내 이마트      167 non-null    int64  
 3   3km내 경쟁 대형마트  167 non-null    int64  
 4   3km내 창고형 마트   167 non-null    int64  
 5   부지 면적         167 non-null    float64
 6   부지 공시지가       167 non-null    int64  
 7   주차 시설         167 non-null    float64
 8   대중 교통         167 non-null    object 
 9   점포형태          167 non-null    int64  
dtypes: float64(2), int64(5), object(3)
memory usage: 13.2+ KB


In [30]:
# emart_all.to_excel("../데이터/emart_all(교통 시설 갯수로 인코딩).xlsx",encoding="cp949",index=False)